In [3]:
import pandas as pd
import numpy as np
import re

# Step 1: Load the Data

df = pd.read_csv("messy_data1.csv")

# Display initial data overview
print("Initial Data Overview:")
display(df.head())
display(df.info())


# Step 2: Inspect the Data

print(df.head())
print(df.info())

# Step 3: Record QA Issues (Export QA Issues to Excel)
qa_issues = []

def log_issue(field, issue, issue_type, line_number, solution, suggestion):
    qa_issues.append({
        "Field": field,
        "QA Issue": issue,
        "Issue Type": issue_type,
        "Data Line Number": line_number,
        "Solution": solution,
        "Suggestion": suggestion
    })

# Step 4: Handle Missing Values
df.dropna(subset=["ID", "Name", "Email", "Join Date"], inplace=True)  # Drop critical missing data
df["Age"].fillna(df["Age"].median(), inplace=True)
df["Salary"].fillna(df["Salary"].mean(), inplace=True)
df["Department"].fillna("Unknown", inplace=True)

# Step 5: Remove Duplicates
df.drop_duplicates(subset=["ID"], keep="first", inplace=True)

# Step 6: Correct Email Formats
def validate_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return bool(re.match(pattern, str(email)))

df = df[df["Email"].apply(validate_email)]

def is_professional_email(email):
    return not any(domain in email for domain in ["gmail.com", "yahoo.com", "hotmail.com"])

df = df[df["Email"].apply(is_professional_email)]

# Step 7: Clean Name Fields
df["Name"] = df["Name"].str.strip().str.title()

# Step 8: Standardize Date Formats
df["Join Date"] = pd.to_datetime(df["Join Date"], errors='coerce')

df.dropna(subset=["Join Date"], inplace=True)  # Remove rows where date conversion failed

# Step 9: Correct Department Names
Department_mapping = {
    "Hr": "HR", "Human Resources": "HR",
    "Eng": "Engineering", "Engg": "Engineering", "Engineer": "Engineering",
    "Mkt": "Marketing", "Mkting": "Marketing",
    "Sal": "Sales", "Sls": "Sales",
    "Supp": "Support", "Cust Support": "Support"
}
df["Department"] = df["Department"].replace(Department_mapping)
#df["department"] = df["department"].replace(Department_mapping)
df["Department"] = df["Department"].str.replace(r'^Sal.*$', 'Sales', regex=True)
df["Department"] = df["Department"].str.replace(r'^HR.*$', 'Human Resource', regex=True)
df["Department"] = df["Department"].str.replace(r'^Eng.*$', 'Engineering', regex=True)
df["Department"] = df["Department"].str.replace(r'^Mark.*$', 'Marketing', regex=True)
df["Department"] = df["Department"].str.replace(r'^Sup.*$', 'Support', regex=True)


# Step 10: Handle Salary Noise
q99 = df["Salary"].quantile(0.99)
df["Salary"] = df["Salary"].clip(upper=q99)

# Export QA Issues to Excel
qa_df = pd.DataFrame(qa_issues)
qa_df.to_excel("QA_Issues_Report.xlsx", index=False)

# Save the Cleaned Data
df.to_csv("cleaned_data.csv", index=False)

print("\nData Cleaning Completed!")

Initial Data Overview:


,Unnamed: 0,ID,Name,Age,Email,Join Date,Salary,Department
0,0,1e407ff9-6255-489d-a0de-34135d4f74bd,Hunter Thomas,25.0,xlopez@hotmail.com,NaN,88552.0,Sales
1,1,379f55b8-87d5-4739-a146-7400b78c24d1,Jeremy Irwin,90.0,Jillian Jenkins,2022-07-07,139227.0,NaN
2,2,18261368-dfa1-47f0-afc6-bddf45926b07,Jennifer Hammondquickly,66.0,jscottgreen.biz,2023-11-21,65550.0,Engineering
3,3,ae7cf7cf-17cf-4c8b-9c44-4f61a9a238e5,Sydney Taylorso,39.0,luke56gonzalez.com,2021-11-05,139932.0,SupportJ
4,4,14ed3e6a-e0f5-4bbe-8d93-8665267f5c90,Julia Lee,71.0,figueroakayla@yahoo.com,NaN,143456.0,Marketing


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11000 non-null  int64  
 1   ID          11000 non-null  object 
 2   Name        8667 non-null   object 
 3   Age         9253 non-null   float64
 4   Email       9731 non-null   object 
 5   Join Date   8808 non-null   object 
 6   Salary      8761 non-null   float64
 7   Department  8745 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 687.6+ KB


None

   Unnamed: 0                                    ID                     Name  \
0           0  1e407ff9-6255-489d-a0de-34135d4f74bd            Hunter Thomas   
1           1  379f55b8-87d5-4739-a146-7400b78c24d1             Jeremy Irwin   
2           2  18261368-dfa1-47f0-afc6-bddf45926b07  Jennifer Hammondquickly   
3           3  ae7cf7cf-17cf-4c8b-9c44-4f61a9a238e5          Sydney Taylorso   
4           4  14ed3e6a-e0f5-4bbe-8d93-8665267f5c90                Julia Lee   

    Age                    Email   Join Date    Salary   Department  
0  25.0       xlopez@hotmail.com         NaN   88552.0        Sales  
1  90.0          Jillian Jenkins  2022-07-07  139227.0          NaN  
2  66.0          jscottgreen.biz  2023-11-21   65550.0  Engineering  
3  39.0       luke56gonzalez.com  2021-11-05  139932.0     SupportJ  
4  71.0  figueroakayla@yahoo.com         NaN  143456.0    Marketing  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 8 c

<ipython-input-3-877a10ead945>:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Age"].fillna(df["Age"].median(), inplace=True)
<ipython-input-3-877a10ead945>:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us